<a href="https://colab.research.google.com/github/sundarp17/semi-supervised/blob/main/train2_svc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:

train2 = pd.read_csv(r'/content/train_data2.csv')
test = pd.read_csv(r'/content/test_data.csv')

In [3]:

train2['Target'] = train2['Target'].replace(['Others'],'Invalid')


In [4]:
train2['Target'].unique()

array(['Invalid', 'Issue', 'Analysis', 'Facts', 'Conclusion',
       'Rule/Law/Holding'], dtype=object)

In [5]:
#cleaning
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_text)

train2_Xtfidf = tfidf_vect.fit_transform(train2['Sentence'])


In [7]:
test['Target']=test['Target'].replace(['Others'],'Invalid')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))

In [8]:
unlabel_1 = test.loc[:100]
print("length of unlabel_1",len(unlabel_1))
unlabel_2 = test.loc[101:200]
print("length of unlabel_2",len(unlabel_2))
unlabel_3 = test.loc[201:300]
print("length of unlabel_3",len(unlabel_3))
unlabel_4 = test.loc[301:400]
print("length of unlabel_4",len(unlabel_4))
unlabel_5 = test.loc[401:]
print("length of unlabel_5",len(unlabel_5))

length of unlabel_1 101
length of unlabel_2 100
length of unlabel_3 100
length of unlabel_4 100
length of unlabel_5 120


In [9]:
unlabel_2.reset_index(inplace=True)
del unlabel_2['index']
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
4,906,agreed accompany gave pawn ticket money reclai...,Facts


In [10]:
unlabel_3.reset_index(inplace=True)
del unlabel_3['index']
unlabel_3.head()

,Unnamed: 0,Sentence,Target
0,1882,find evidence amply sufficient sustain juryâs ...,Analysis
1,964,actions show thatâ,Invalid
2,1417,certification attending physician shown filed ...,Facts
3,2932,testified agent bland first left automobile wa...,Facts
4,1584,officer testified could smell odor alcohol app...,Facts


In [11]:
unlabel_4.reset_index(inplace=True)
del unlabel_4['index']
unlabel_4.head()

,Unnamed: 0,Sentence,Target
0,266,section 6 pl,Invalid
1,3469,amended answer appellant alleged bond invalid ...,Analysis
2,1471,evidence sufficient support conviction appella...,Analysis
3,1990,cases cited note 19,Invalid
4,2757,july 8 1960 order entered revoking said probat...,Facts


In [12]:
unlabel_5.reset_index(inplace=True)
del unlabel_5['index']
unlabel_5.head()

,Unnamed: 0,Sentence,Target
0,2979,overrule appellantâs contention court erred pe...,Analysis
1,2135,state regarded authority case,Invalid
2,1591,testimony adduced appellant mother lillie mae ...,Facts
3,658,appellant needle marks inside left elbow,Facts
4,2562,analysis contents 3 capsules chemist kenneth a...,Facts


In [13]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.calibration import CalibratedClassifierCV


X_train, x_val, Y_train, y_val = train_test_split(train2_Xtfidf,train2['Target'],test_size=0.26,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model=calibrator.fit(X_train, Y_train)
pred_svm = model.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6692477876106194
                  precision    recall  f1-score   support

        Analysis       0.52      0.39      0.44       126
      Conclusion       0.69      0.56      0.62        45
           Facts       0.74      0.90      0.82       480
         Invalid       0.47      0.43      0.45       132
           Issue       0.51      0.43      0.47        51
Rule/Law/Holding       0.73      0.27      0.40        70

        accuracy                           0.67       904
       macro avg       0.61      0.50      0.53       904
    weighted avg       0.66      0.67      0.65       904



In [14]:
x_un1 = tfidf_vect.transform(unlabel_1['Sentence'])
pred_unlabel_1 = model.predict_proba(x_un1)

In [15]:
import numpy as np
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_1:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 4, 2, 2, 2, 2, 2, 0, 2, 2, 1, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 3]
[0.9947364811288171, 0.995350562656587, 0.9999246668177597, 0.9805267158180578, 0.9986327268271624, 0.9967933544112038, 0.997190902272551, 0.9979384279055198, 0.993387060006523, 0.9815105608712342, 0.9847802278432465, 0.9990275157459144, 0.9950735841822161, 0.9984069498019716, 0.9891576673310161, 0.9994406019934379, 0.9829746744708211, 0.9811032653545615, 0.986629040985706, 0.9934883787477062, 0.9915253251412279, 0.9877614873564897, 0.9992345410967807, 0.9996316626148171, 0.9944006400835711, 0.9957919114608734, 0.99567512472169, 0.9960121696848192, 0.991054536187153, 0.9962678497937677, 0.9977987015654446, 0.9993190163084229, 0.9966567648646351, 0.9878595211576685, 0.9802906149545654, 0.9998712587989534, 0.9873745374767408, 0.9825189430801107, 0.9927254367134486, 0.9953991542457207, 0.9955964082156826, 0.9899499435869533, 0.999296131939602, 0.996512

In [16]:
unlabel_1=unlabel_1.loc[[1, 4, 5, 6, 11, 12, 17, 18, 19, 20, 21, 23, 24, 29, 30, 31, 33, 35, 37, 39, 40, 41, 45, 48, 49, 50, 51, 52, 55, 56, 60, 62, 68, 70, 75, 76, 77, 78, 79, 84, 87, 89, 94, 95, 96, 98],:]
unlabel_1.head()

,Unnamed: 0,Sentence,Target
1,466,additional moral justification may due complai...,Analysis
4,79,eligible leaves renew contacts family,Facts
5,1955,got downstairs ponce went car wife left home,Facts
6,324,defendant argues impossible reconcile plaintif...,Facts
11,1604,police notified deceased found lying mrs moers...,Facts


In [17]:
frame_1 = [train2,unlabel_1]
train2_1 = pd.concat(frame_1)
len(train2_1)

3522

In [18]:
x_train_1 = tfidf_vect.transform(train2_1['Sentence'])
x_train_1.shape
X_train, x_val, Y_train, y_val = train_test_split(x_train_1,train2_1['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_1=calibrator.fit(X_train, Y_train)
pred_svm = model_1.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.675177304964539
                  precision    recall  f1-score   support

        Analysis       0.57      0.46      0.51        94
      Conclusion       0.68      0.50      0.58        38
           Facts       0.74      0.90      0.81       357
         Invalid       0.56      0.49      0.52       121
           Issue       0.64      0.41      0.50        44
Rule/Law/Holding       0.50      0.29      0.37        51

        accuracy                           0.68       705
       macro avg       0.61      0.51      0.55       705
    weighted avg       0.66      0.68      0.66       705



In [19]:
x_un2 = tfidf_vect.transform(unlabel_2['Sentence'])
pred_unlabel_2 = model_1.predict_proba(x_un2)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_2:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 3, 2, 1, 2, 1, 2, 0, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2]
[0.9994590905224398, 0.9984093082884145, 0.9923361073097062, 0.9988604711437876, 0.9999708656206002, 0.9877145565086215, 0.99480018177729, 0.9932939986224993, 0.9921369624188255, 0.9982672479856253, 0.981980175155705, 0.9991172717714409, 0.9997244313565374, 0.9993457707283452, 0.9980301764053984, 0.987312043989234, 0.9985927919712368, 0.9854918649509639, 0.988563528955601, 0.9997962898464204, 0.9835709172828104, 0.9997154786053813, 0.9998943929021143, 0.9993113805361764, 0.9988279883976957, 0.9989106735464507, 0.9828610742709933, 0.9962936322446234, 0.9970905760915264, 0.9950560132388693, 0.9996459959875575, 0.9966851078822191, 0.9965640536147065, 0.9900639546606846, 0.9920617715006134]
[2, 4, 6, 8, 12, 15, 20, 21, 24, 29, 31, 36, 37, 39, 42, 44, 49, 50, 57, 59, 62, 63, 64, 69, 70, 71, 75, 83, 85, 86, 88, 90, 91, 95, 99]
35
35
35


In [20]:
unlabel_2 = unlabel_2.loc[[2, 4, 6, 8, 12, 15, 20, 21, 24, 29, 31, 36, 37, 39, 42, 44, 49, 50, 57, 59, 62, 63, 64, 69, 70, 71, 75, 83, 85, 86, 88, 90, 91, 95, 99],:]
unlabel_2.head()

,Unnamed: 0,Sentence,Target
2,2072,appellant testify offer evidence behalf,Facts
4,906,agreed accompany gave pawn ticket money reclai...,Facts
6,570,appellant performed work asked 7500 money ther...,Facts
8,710,appellant indicted robbery indictment alleging...,Analysis
12,2085,mrs joyce wisnoski sister appellant testified ...,Facts


In [21]:
frame_2 = [train2_1,unlabel_2]
train2_2 = pd.concat(frame_2)
len(train2_2)

3557

In [22]:
x_train_2 = tfidf_vect.transform(train2_2['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_2,train2_2['Target'],test_size=0.27,random_state=2)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_2=calibrator.fit(X_train, Y_train)
pred_svm = model_2.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6482830385015609
                  precision    recall  f1-score   support

        Analysis       0.42      0.44      0.43       142
      Conclusion       0.75      0.45      0.56        47
           Facts       0.73      0.90      0.81       501
         Invalid       0.48      0.40      0.43       149
           Issue       0.79      0.25      0.38        60
Rule/Law/Holding       0.62      0.26      0.36        62

        accuracy                           0.65       961
       macro avg       0.63      0.45      0.50       961
    weighted avg       0.64      0.65      0.63       961



In [23]:
x_un3 = tfidf_vect.transform(unlabel_3['Sentence'])
pred_unlabel_3 = model_2.predict_proba(x_un3)

pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_3:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 3, 1, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2]
[0.9939777846530989, 0.9892932204444125, 0.991178254537783, 0.9986894678507207, 0.999055304008049, 0.9993174394506545, 0.9861101082206524, 0.9996840006841463, 0.9908635372019093, 0.9843149208712303, 0.9990923613776207, 0.985372305094474, 0.9952491265587107, 0.9847692281004353, 0.997995567120525, 0.9984028607366648, 0.9937122150693107, 0.997329561123196, 0.9840460632735549, 0.9904430920325825, 0.9937224647388999, 0.9910111406029575, 0.9880432870562146, 0.98764707071928, 0.9843992286019593, 0.9854671219109835, 0.9984708704181482, 0.99987320625809, 0.9851000918029763, 0.995102801889305, 0.9988656943881957, 0.9974127633786048, 0.9997396567533058, 0.9869828696761523, 0.9975070921490992, 0.984240306826718, 0.9890912810397853, 0.9995809018212438, 0.9926103252271362]
[0, 2, 3, 7, 8, 12, 19, 20, 23, 26, 27, 29, 30, 33, 36, 43, 46, 47, 48, 49, 51, 52, 53, 54, 56, 61, 63, 65, 66, 

In [24]:
unlabel_3=unlabel_3.loc[[0, 2, 3, 7, 8, 12, 19, 20, 23, 26, 27, 29, 30, 33, 36, 43, 46, 47, 48, 49, 51, 52, 53, 54, 56, 61, 63, 65, 66, 67, 68, 70, 72, 73, 75, 79, 85, 89, 91],:]
frame_3 = [train2_2,unlabel_3]
train2_3 = pd.concat(frame_3)

x_train_3 = tfidf_vect.transform(train2_3['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_3,train2_3['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_3=calibrator.fit(X_train, Y_train)
pred_svm = model_3.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6666666666666666
                  precision    recall  f1-score   support

        Analysis       0.50      0.45      0.47       101
      Conclusion       0.70      0.62      0.66        48
           Facts       0.74      0.88      0.81       368
         Invalid       0.49      0.42      0.45       115
           Issue       0.77      0.48      0.59        42
Rule/Law/Holding       0.48      0.26      0.34        46

        accuracy                           0.67       720
       macro avg       0.61      0.52      0.55       720
    weighted avg       0.65      0.67      0.65       720



In [25]:
x_un4 = tfidf_vect.transform(unlabel_4['Sentence'])
pred_unlabel_4 = model_3.predict_proba(x_un4)


pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_4:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 2, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 3, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 5, 2, 2, 2]
[0.9995152682435181, 0.9819606147605406, 0.9909929855487182, 0.9971755331703275, 0.9911641754638728, 0.999189801372556, 0.9982650230622059, 0.9867787580148596, 0.9855492002027004, 0.9995917558692478, 0.9959354136116574, 0.9978796753948034, 0.9898722956827486, 0.9987796218260483, 0.9988991523901113, 0.9986611740364049, 0.9963799537561194, 0.9957509857702126, 0.9935251008748918, 0.9995207748490008, 0.9982114162946774, 0.9968407916828781, 0.9997857145280881, 0.9928956657621738, 0.9878157001594845, 0.9871270073050724, 0.9957273354657478, 0.99204376513621, 0.9974615354424395, 0.9940105993809516, 0.9931048384657246, 0.9919422005693842, 0.9811356422050811, 0.9928699511232212, 0.9974945210534429, 0.9990423610789205, 0.997677490477019, 0.9999596671358464, 0.9965015152064461, 0.997986684117922, 0.9901805728586691, 0.9833740177242989, 0.9887572136355363, 0.989369

In [26]:
unlabel_4=unlabel_4.loc[[2, 4, 6, 7, 8, 10, 15, 16, 17, 18, 21, 22, 25, 27, 28, 29, 30, 31, 32, 34, 37, 38, 40, 42, 46, 49, 50, 55, 56, 57, 58, 60, 61, 64, 67, 70, 71, 75, 80, 85, 87, 94, 96, 97, 99 ],:]

In [27]:
frame_4 = [train2_3,unlabel_4]
train2_4 = pd.concat(frame_4)
len(train2_4)
x_train_4 = tfidf_vect.transform(train2_4['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_4,train2_4['Target'],test_size=0.27,random_state=42)
support = svm.LinearSVC()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_4=calibrator.fit(X_train, Y_train)
pred_svm = model_4.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.649390243902439
                  precision    recall  f1-score   support

        Analysis       0.52      0.40      0.45       153
      Conclusion       0.55      0.42      0.48        50
           Facts       0.73      0.89      0.81       503
         Invalid       0.48      0.39      0.43       161
           Issue       0.49      0.40      0.44        55
Rule/Law/Holding       0.57      0.37      0.45        62

        accuracy                           0.65       984
       macro avg       0.56      0.48      0.51       984
    weighted avg       0.63      0.65      0.63       984



In [28]:
x_un5 = tfidf_vect.transform(unlabel_5['Sentence'])

pred_unlabel_5 = model_4.predict_proba(x_un5)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_5:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 2, 2, 3, 2, 5, 3, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0]
[0.9835706534737136, 0.9986913232915438, 0.9967103220665957, 0.9933960895698312, 0.9867153797042187, 0.9859394256773638, 0.9973931304080755, 0.9829850424710682, 0.9972856627810629, 0.9993490248048491, 0.9874873179747494, 0.986386682212982, 0.9968721730753612, 0.9997345255262626, 0.9853990134436327, 0.9933317591407834, 0.9994710917275414, 0.9962158946034665, 0.9914969666300626, 0.9992290080284413, 0.9976601318536596, 0.9994525724668963, 0.9994821151872488, 0.9958230562471185, 0.9856995193413235]
[1, 3, 7, 8, 12, 18, 19, 22, 23, 26, 27, 28, 32, 36, 46, 52, 53, 60, 65, 82, 86, 94, 95, 97, 103]
25
25
25


In [29]:
unlabel_5 = unlabel_5.loc[[1, 3, 7, 8, 12, 18, 19, 22, 23, 26, 27, 28, 32, 36, 46, 52, 53, 60, 65, 82, 86, 94, 95, 97, 103],:]

In [30]:
frame_5 = [train2_4,unlabel_5]
train2_5 = pd.concat(frame_5)
len(train2_5)
x_train_5 = tfidf_vect.transform(train2_5['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_5,train2_5['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_5=calibrator.fit(X_train, Y_train)
pred_svm = model_5.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6730245231607629
                  precision    recall  f1-score   support

        Analysis       0.45      0.44      0.44        95
      Conclusion       0.69      0.52      0.59        42
           Facts       0.78      0.87      0.82       382
         Invalid       0.48      0.45      0.47       108
           Issue       0.63      0.47      0.54        51
Rule/Law/Holding       0.59      0.41      0.48        56

        accuracy                           0.67       734
       macro avg       0.60      0.53      0.56       734
    weighted avg       0.66      0.67      0.66       734



In [31]:
t_p = tfidf_vect.transform(test['Sentence'])
test_pred = model_5.predict(t_p)
print('Accuracy %s' % accuracy_score(test_pred,test['Target']))

Accuracy 0.6717850287907869


In [32]:
from sklearn.metrics import precision_recall_fscore_support
print(precision_recall_fscore_support(test['Target'],test_pred,average='macro'))

(0.6494647019647021, 0.5908487567682178, 0.6051923271375125, None)
